In [0]:
## Laboratorio donde se realizan practicas de limpiezas de datos y consultas Basicas de datos en Pyspark

In [0]:
sc=spark.sparkContext
import re
from pyspark.sql.types import IntegerType,DoubleType
from pyspark.sql.functions import regexp_replace,col, avg, min, max, count,desc,sum,asc,row_number,when
from pyspark.sql import Window

In [0]:
price_2022=spark.read.option('header','true').option('inferschema','true').csv('/FileStore/tables/prices_2022.csv')
avg_speed=spark.read.option('header','true').option('inferschema','true').csv('/FileStore/tables/avg_speed.csv')
users=spark.read.option('header','true').option('inferschema','true').csv('/FileStore/tables/users.csv')

In [0]:
users.show()

+---------------+------------------+--------+--------------+-------------+
|Country or area|         Subregion|  Region|Internet users|   Population|
+---------------+------------------+--------+--------------+-------------+
|          China|      Eastern Asia|    Asia| 1,010,740,000|1,427,647,786|
|          India|     Southern Asia|    Asia|   833,710,000|1,352,642,280|
|  United States|  Northern America|Americas|   312,320,000|  327,096,265|
|      Indonesia|South-eastern Asia|    Asia|   196,000,000|  267,670,543|
|         Brazil|     South America|Americas|   160,010,801|  209,469,323|
|        Nigeria|    Western Africa|  Africa|   136,203,231|  195,874,685|
|     Bangladesh|     Southern Asia|    Asia|   129,180,000|  166,303,498|
|         Russia|    Eastern Europe|  Europe|   124,000,000|  145,734,038|
|       Pakistan|     Southern Asia|    Asia|   118,800,000|  213,756,286|
|          Japan|      Eastern Asia|    Asia|   117,400,000|  127,202,192|
|         Mexico|  Northe

In [0]:
for col in price_2022.columns:
  price_2022=price_2022.withColumnRenamed(col,re.sub(r'[.\-\$\()–]', '',col.lower().strip().replace(" ", "_").replace("__", "_")))## el re.sub(r'caracacteres a seleccionar,caracterque remplaza,dato al cual aplica la transformacion. en este caso al col,lower.strip.replace)


In [0]:
for col in avg_speed.columns:
  avg_speed=avg_speed.withColumnRenamed(col, col.lower().strip().replace(" ", "_"))


In [0]:
for col in users.columns:
  users=users.withColumnRenamed(col, col.lower().strip().replace(" ", "_"))

In [0]:
from pyspark.sql.functions import regexp_replace,col
users = users.withColumn("internet_users",regexp_replace(col("internet_users"), ",", "").cast(IntegerType()))
users = users.withColumn("population",regexp_replace(col("population"), ",", "").cast(IntegerType()))
#users=users.withColumn("internet_users", users["internet_users"].cast(IntegerType()))

In [0]:
price_2022.show()

+------------+------------------+--------------------+--------------------+------------------------+----------------------------+----------------------+---------------------------------------------+------------------------------------------+
|country_code|              name|  continental_region|no_of_internet_plans|average_price_of_1gb_usd|cheapest_1gb_for_30_days_usd|most_expensive_1gb_usd|average_price_of_1gb_usd_at_the_start_of_2021|average_price_of_1gb_usd__at_start_of_2020|
+------------+------------------+--------------------+--------------------+------------------------+----------------------------+----------------------+---------------------------------------------+------------------------------------------+
|          IL|            Israel|           NEAR EAST|                  27|                   $0.05|                       $0.02|                $20.95|                                        $0.11|                                     $0.90|
|          KG|        Kyrgyzstan

In [0]:
avg_speed.show()

+--------------------+------+
|             country|   avg|
+--------------------+------+
|      (Mbit/s)Ookla"|  null|
|              Canada| 75.18|
|         South Korea|117.95|
|         Netherlands|108.33|
|               Japan| 44.05|
|              Norway|134.73|
|           Singapore| 67.99|
|           Australia| 76.52|
|         Switzerland| 82.29|
|             Belgium| 58.65|
|             Croatia| 86.48|
|         New Zealand| 49.49|
|             Austria|  56.6|
|             Denmark|105.65|
|           Lithuania| 50.13|
|      Czech Republic| 44.55|
|United Arab Emirates|135.35|
|             Hungary| 41.32|
|               Qatar|120.69|
|     North Macedonia| 64.23|
+--------------------+------+
only showing top 20 rows



In [0]:
print("Esquema de AVG_SPEED\n")
avg_speed.printSchema()
print("Esquema de price_2022\n")
price_2022.printSchema()
print("Esquema de users\n")
users.printSchema()

Esquema de AVG_SPEED

root
 |-- country: string (nullable = true)
 |-- avg: double (nullable = true)

Esquema de price_2022

root
 |-- country_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- continental_region: string (nullable = true)
 |-- no_of_internet_plans: integer (nullable = true)
 |-- average_price_of_1gb_usd: string (nullable = true)
 |-- cheapest_1gb_for_30_days_usd: string (nullable = true)
 |-- most_expensive_1gb_usd: string (nullable = true)
 |-- average_price_of_1gb_usd_at_the_start_of_2021: string (nullable = true)
 |-- average_price_of_1gb_usd__at_start_of_2020: string (nullable = true)

Esquema de users

root
 |-- country_or_area: string (nullable = true)
 |-- subregion: string (nullable = true)
 |-- region: string (nullable = true)
 |-- internet_users: integer (nullable = true)
 |-- population: integer (nullable = true)



In [0]:
from pyspark.sql.functions import col, avg, min, max, count,desc,sum,asc

In [0]:
top_users_america=users.select(
col('country_or_area').alias("Pais"),
col('region'),
col('subregion'),
col('internet_users').alias('usuarios_internet')
).where(col('region')=='Americas').orderBy(desc('internet_users'))

In [0]:
top_users_regiones=users.groupBy('region').agg(
  sum(col('internet_users')).alias('usuarios_internet')
).orderBy(desc('usuarios_internet'))

In [0]:
window_spec = Window.partitionBy("region").orderBy(desc("internet_users")) ## crea  ventanas que ordena por internet use y resetea cuando cambia de region
user_rank_po_region = users.withColumn("rank", row_number().over(window_spec))## creo una columna que hace un ranking reseteando por la ventana (por regiones) 


In [0]:
users_max_porregion_pais = user_rank_po_region.where(col("rank")==1).drop("rank")##toma el rank 1 

In [0]:
users_max_porregion_pais_reparticionado=users_max_porregion_pais.repartition(3)

In [0]:
users_max_porregion_pais_reparticionado.write.partitionBy("region").format("avro").mode("overwrite").save("/FileStore/ProyectoFinal/salida")

In [0]:
users_max_porregion_pais_reparticionado \
    .write \
    .partitionBy("region") \
    .format("avro") \
    .mode("overwrite") \
    .save("/FileStore/ProyectoFinal/salida")##el slash periite hacer saltos de linea en el codigo ... interesante

In [0]:
dbutils.fs.ls("/FileStore/ProyectoFinal/salida")

Out[18]: [FileInfo(path='dbfs:/FileStore/ProyectoFinal/salida/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1731287444000),
 FileInfo(path='dbfs:/FileStore/ProyectoFinal/salida/_committed_863615927311684940', name='_committed_863615927311684940', size=35, modificationTime=1731281137000),
 FileInfo(path='dbfs:/FileStore/ProyectoFinal/salida/region=Africa/', name='region=Africa/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/ProyectoFinal/salida/region=Americas/', name='region=Americas/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/ProyectoFinal/salida/region=Asia/', name='region=Asia/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/ProyectoFinal/salida/region=Europe/', name='region=Europe/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/ProyectoFinal/salida/region=Oceania/', name='region=Oceania/', size=0, modificationTime=0)]

In [0]:
avg_speed.join(users,col('country_or_area')==col('country')).select(
  col('*')
  ).orderBy(desc('avg')).show(10)
         

+--------------------+------+--------------------+---------------+------+--------------+----------+
|             country|   avg|     country_or_area|      subregion|region|internet_users|population|
+--------------------+------+--------------------+---------------+------+--------------+----------+
|United Arab Emirates|135.35|United Arab Emirates|   Western Asia|  Asia|       8913217|   9630959|
|              Norway|134.73|              Norway|Northern Europe|Europe|       5120225|   5337962|
|               Qatar|120.69|               Qatar|   Western Asia|  Asia|       2532059|   2781682|
|         South Korea|117.95|         South Korea|   Eastern Asia|  Asia|      49421084|  51171706|
|         Netherlands|108.33|         Netherlands| Western Europe|Europe|      15877494|  17059560|
|             Denmark|105.65|             Denmark|Northern Europe|Europe|       5407278|   5752126|
|        Saudi Arabia|102.79|        Saudi Arabia|   Western Asia|  Asia|      27048861|  33702756|


In [0]:
users.show()

+---------------+------------------+--------+--------------+----------+
|country_or_area|         subregion|  region|internet_users|population|
+---------------+------------------+--------+--------------+----------+
|          China|      Eastern Asia|    Asia|    1010740000|1427647786|
|          India|     Southern Asia|    Asia|     833710000|1352642280|
|  United States|  Northern America|Americas|     312320000| 327096265|
|      Indonesia|South-eastern Asia|    Asia|     196000000| 267670543|
|         Brazil|     South America|Americas|     160010801| 209469323|
|        Nigeria|    Western Africa|  Africa|     136203231| 195874685|
|     Bangladesh|     Southern Asia|    Asia|     129180000| 166303498|
|         Russia|    Eastern Europe|  Europe|     124000000| 145734038|
|       Pakistan|     Southern Asia|    Asia|     118800000| 213756286|
|          Japan|      Eastern Asia|    Asia|     117400000| 127202192|
|         Mexico|  Northern America|Americas|      92010000| 128

In [0]:
columns_to_transform = [
    "average_price_of_1gb_usd", 
    "cheapest_1gb_for_30_days_usd", 
    "average_price_of_1gb_usd_at_the_start_of_2021",
    "average_price_of_1gb_usd__at_start_of_2020",
    "most_expensive_1gb_usd"
]

In [0]:
for columna in columns_to_transform:
  price_2022 = price_2022.withColumn(columna,regexp_replace(col(columna), r"\$", "").cast(DoubleType()))

In [0]:
price_2022.printSchema()

root
 |-- country_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- continental_region: string (nullable = true)
 |-- no_of_internet_plans: integer (nullable = true)
 |-- average_price_of_1gb_usd: double (nullable = true)
 |-- cheapest_1gb_for_30_days_usd: double (nullable = true)
 |-- most_expensive_1gb_usd: double (nullable = true)
 |-- average_price_of_1gb_usd_at_the_start_of_2021: double (nullable = true)
 |-- average_price_of_1gb_usd__at_start_of_2020: double (nullable = true)



In [0]:
price_2022.join(users,col("country_or_area")==col("name")).select(
  col("name").alias("country"),
  col("average_price_of_1gb_usd_at_the_start_of_2021").alias("avg_price_1gb_internet"),
  col("subregion"),
  col("internet_users"),
  col("region")
).groupBy(col("region")).agg(
  avg("avg_price_1gb_internet")
).withColumn("clasificar_region",when(col("region").startswith("A"),"region_a")
                                .when(col("region").startswith("E"),"region_e")
                                .otherwise("other")).orderBy(asc(col("region"))).show()

+--------+---------------------------+-----------------+
|  region|avg(avg_price_1gb_internet)|clasificar_region|
+--------+---------------------------+-----------------+
|  Africa|          5.692040816326531|         region_a|
|Americas|          6.772499999999998|         region_a|
|    Asia|         2.9502325581395343|         region_a|
|  Europe|          3.050227272727273|         region_e|
| Oceania|          7.677142857142857|            other|
+--------+---------------------------+-----------------+



---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-2993261745819362> in <module>
----> 1 users.na.fill(col("average_price_of_1gb_usd"),0).show()

/databricks/spark/python/pyspark/sql/dataframe.py in fill(self, value, subset)
   2996 
   2997     def fill(self, value, subset=None):
-> 2998         return self.df.fillna(value=value, subset=subset)
   2999 
   3000     fill.__doc__ = DataFrame.fillna.__doc__

/databricks/spark/python/pyspark/sql/dataframe.py in fillna(self, value, subset)
   2227         """
   2228         if not isinstance(value, (float, int, str, bool, dict)):
-> 2229             raise TypeError("value should be a float, int, string, bool or dict")
   2230 
   2231         # Note that bool validates isinstance(int), but we don't want to

TypeError: value should be a float, int, string, bool or dict